In [ ]:
Documentação:
Decisões Técnicas:
Python: Escolhido pela facilidade de uso e pela vasta quantidade de bibliotecas disponíveis para manipulação de dados.
SQL Server: Escolhido pela robustez e suporte a operações complexas de banco de dados.
Dash: Escolhido pela capacidade de criar visualizações interativas de maneira rápida e eficaz.


Segurança:
Utilização de conexões seguras com as APIs e o banco de dados.
Sanitização dos dados de entrada para evitar injeção de SQL.

Escalabilidade:
Arquitetura modular permitindo fácil expansão dos módulos de ETL.
Utilização de transações e operações em lote para otimizar o carregamento de dados.


In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import dash
from dash import dcc, html
import plotly.express as px
from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente
load_dotenv()

# Configurações
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')
TRAFFIC_API_KEY = os.getenv('TRAFFIC_API_KEY')
DATABASE_URL = os.getenv('DATABASE_URL')

Base = declarative_base()

class Weather(Base):
    __tablename__ = 'weather'
    id = Column(Integer, primary_key=True)
    location = Column(String)
    temperature = Column(Float)
    humidity = Column(Float)
    date = Column(DateTime)

class Traffic(Base):
    __tablename__ = 'traffic'
    id = Column(Integer, primary_key=True)
    origin = Column(String)
    destination = Column(String)
    duration = Column(Integer)
    distance = Column(Float)
    date = Column(DateTime)

# Funções de Extração
def fetch_weather_data(api_key, location='London'):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json()

def fetch_traffic_data(api_key, origin='New York', destination='Los Angeles'):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={api_key}"
    response = requests.get(url)
    return response.json()

# Funções de Limpeza e Transformação
def clean_weather_data(raw_data):
    df = pd.json_normalize(raw_data)
    df.drop_duplicates(inplace=True)
    df['date'] = pd.to_datetime(df['dt'], unit='s')
    df.fillna(method='ffill', inplace=True)
    cleaned_data = df[['name', 'main.temp', 'main.humidity', 'date']]
    cleaned_data.columns = ['location', 'temperature', 'humidity', 'date']
    return cleaned_data

def clean_traffic_data(raw_data):
    routes = raw_data.get('routes', [])
    if not routes:
        return pd.DataFrame()
    
    df = pd.json_normalize(routes, 'legs')
    df.drop_duplicates(inplace=True)
    df['date'] = pd.to_datetime(df['start_time.value'], unit='s')
    df.fillna(method='ffill', inplace=True)
    cleaned_data = df[['start_address', 'end_address', 'duration.value', 'distance.value', 'date']]
    cleaned_data.columns = ['origin', 'destination', 'duration', 'distance', 'date']
    return cleaned_data

# Integração com Banco de Dados
def load_data_to_db(engine, data, table_class):
    Session = sessionmaker(bind=engine)
    session = Session()
    session.bulk_insert_mappings(table_class, data.to_dict(orient="records"))
    session.commit()
    session.close()

engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)

# Extração e Transformação dos Dados
weather_raw = fetch_weather_data(WEATHER_API_KEY)
weather_data = clean_weather_data(weather_raw)

traffic_raw = fetch_traffic_data(TRAFFIC_API_KEY)
traffic_data = clean_traffic_data(traffic_raw)

# Carregamento dos Dados
load_data_to_db(engine, weather_data, Weather)
load_data_to_db(engine, traffic_data, Traffic)

# Visualização de Dados
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dados de Clima e Trânsito'),
    dcc.Graph(id='weather-graph'),
    dcc.Graph(id='traffic-graph')
])

@app.callback(
    [dash.dependencies.Output('weather-graph', 'figure'),
     dash.dependencies.Output('traffic-graph', 'figure')],
    [dash.dependencies.Input('interval-component', 'n_intervals')]
)
def update_graphs(n):
    weather_df = pd.read_sql('weather', engine)
    traffic_df = pd.read_sql('traffic', engine)
    
    weather_fig = px.line(weather_df, x='date', y='temperature', title='Temperatura ao longo do tempo')
    traffic_fig = px.line(traffic_df, x='date', y='duration', title='Duração do Tráfego ao longo do tempo')
    
    return weather_fig, traffic_fig

if __name__ == '__main__':
    app.run_server(debug=True)
